In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 10
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    '''
    SELECT 
    count(CRIM) as CRIM,
    count(CHAS) as CHAS,
    count(ZN) as ZN,
    count(INDUS) as INDUS,
    count(NOX) as NOX
    FROM boston
    where CRIM is NULL or CHAS is NULL or ZN is NULL or INDUS is NULL or NOX is NULL
    ''',
    con,
)

,CRIM,CHAS,ZN,INDUS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    '''
    SELECT
    count(DISTINCT CRIM) as CRIM,
    count(DISTINCT CHAS) as CHAS,
    count(DISTINCT ZN) as ZN,
    count(DISTINCT INDUS) as INDUS,
    count(DISTINCT NOX) as NOX
    FROM boston
    
    limit 5
    ''',
    con,
)

,CRIM,CHAS,ZN,INDUS,NOX
0,504,2,26,76,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [11]:
pd.read_sql(
    '''
    SELECT * FROM 
    (    
    SELECT 'CHAS' as name, MIN(CHAS) as memi_val FROM boston
    UNION ALL
    SELECT name, AVG(CHAS) as memi_val FROM (
    SELECT CHAS, 'CHAS' as name
    FROM boston order by CHAS
    limit 2 - (SELECT COUNT(*) FROM boston) % 2
    offset (SELECT COUNT(*) - 1 FROM boston) / 2)
    
    UNION

    SELECT  'INDUS' as name, MIN(INDUS) as memi_val FROM boston
    UNION
    SELECT name, AVG(INDUS) as memi_val FROM (
    SELECT INDUS, 'INDUS' as name
    FROM boston order by INDUS
    limit 2 - (SELECT COUNT(*) FROM boston) % 2
    offset (SELECT COUNT(*) - 1 FROM boston) / 2)
    
    UNION

    SELECT  'ZN' as name, MIN(ZN) as memi_val FROM boston
    UNION
    SELECT name, AVG(ZN) as memi_val FROM (
    SELECT ZN, 'ZN' as name
    FROM boston order by ZN
    limit 2 - (SELECT COUNT(*) FROM boston) % 2
    offset (SELECT COUNT(*) - 1 FROM boston) / 2)
    
    UNION
    
    SELECT 'NOX' as name, MIN(NOX) as memi_val FROM boston
    UNION
    SELECT name, AVG(NOX) as memi_val FROM (
    SELECT NOX, 'NOX' as name
    FROM boston order by NOX
    limit 2 - (SELECT COUNT(*) FROM boston) % 2
    offset (SELECT COUNT(*) - 1 FROM boston) / 2)
    
    UNION
    
    SELECT  'CRIM' as name, MIN(CRIM) as memi_val FROM boston
    UNION
    SELECT name, AVG(CRIM) as memi_val FROM (
    SELECT CRIM, 'CRIM' as name
    FROM boston order by CRIM
    limit 2 - (SELECT COUNT(*) FROM boston) % 2
    offset (SELECT COUNT(*) - 1 FROM boston) / 2)
    )
    WHERE memi_val = 0
    ''',
    con,
)

,name,memi_val
0,CHAS,0.0
1,ZN,0.0


В среднем уровень преступности низкий для округа жилья не граничещего с рекой

4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [14]:
pd.read_sql(
    '''
    select LSTAT, MEDV as 'стоимость',
        dense_rank() over (partition by LSTAT order by MEDV desc) as 'ранг'
    from boston
    order by ранг asc
    --limit 10
    ''',
    con,
)

,LSTAT,стоимость,ранг
0,1.73,50.0,1
1,1.92,50.0,1
2,1.98,34.9,1
3,2.47,41.7,1
4,2.87,36.4,1
...,...,...,...
501,30.81,7.2,2
502,6.36,23.1,3
503,7.79,18.6,3
504,8.05,18.2,3


Вывод: Жилье дороже там, где меньше % населения с более низким статусом, 

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [16]:
pd.read_sql(
    '''
    select distinct CHAS,
        avg(MEDV) OVER (partition by CHAS) as 'avg_price'
    FROM boston
    ''',
    con,
)

,CHAS,avg_price
0,0.0,22.093843
1,1.0,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [74]:
pd.read_sql(
    '''
    select CHAS, 
    dif_INDUS as INDUS,
    dif_NOX as NOX
    from
     (
          select CHAS,
          last_value(mean_CRIM) over (order by mean_CRIM) - first_value(mean_CRIM) over (order by CHAS)  as dif_CRIM,
          last_value(mean_ZN) over (order by mean_ZN) - first_value(mean_ZN) over (order by CHAS)  as dif_ZN, 
          last_value(mean_INDUS) over (order by mean_INDUS) - first_value(mean_INDUS) over (order by CHAS)  as dif_INDUS,
          last_value(mean_NOX) over (order by mean_NOX) - first_value(mean_NOX) over (order by CHAS)  as dif_NOX
         from
            (
                SELECT DISTINCT CHAS,    
                avg(CRIM) over(partition by CHAS) as mean_CRIM,
                avg(ZN) over(partition by CHAS) as mean_ZN,
                avg(INDUS) over(partition by CHAS) as mean_INDUS,
                avg(NOX) over(partition by CHAS) as mean_NOX
                FROM boston
            ) a
    ) 
    where chas = 1.0 

    ''',
    con,
)

,CHAS,INDUS,NOX
0,1.0,1.69995,0.041609


Вывод: Возле реки повышена концентрация азота (возможно за счет речного транспорта).

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [36]:
pd.read_sql(
    '''
    select distinct a.PCT, INDUS, NOX
    from
    (SELECT distinct INDUS, NOX, 
    (ntile(100/10) over (order by INDUS))*10 as 'PCT'
    FROM boston
    order by pct) as a
    group by a.PCT
    ''',
    con,
)

,PCT,INDUS,NOX
0,10,0.46,0.422
1,20,2.93,0.401
2,30,4.39,0.442
3,40,5.96,0.499
4,50,7.38,0.493
5,60,9.69,0.585
6,70,13.89,0.550
7,80,18.10,0.693
8,90,18.10,0.740
9,100,19.58,0.871


Вывод: Чем выше промышленная застройка тем выше концентрация азота